# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

#### Created by Silvia Huang, CFFS Data Analyst
****

## Part II: Data Cleaning

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

In [3]:
# Set the root path, change the the current working directory into the project folder
path = "/Users/silvia/cffs-label"
os.chdir(path)

In [4]:
# Enable reading data table in the scrolling window if you prefer
#pd.set_option("display.max_rows", None, "display.max_columns", None)

***

## Import Preprocessed Datasets

In [5]:
# Read Items_List.csv
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [6]:
Items.head()

ItemId                  Description  CaseQty CaseUOM  PakQty PakUOM  \
0   I-4271          APPLES GRANNY SMITH    113.0      ea     1.0     CT   
1   I-4971  ARTICHOKE 1/4 SALAD CUT TFC      6.0  LG CAN     2.5     Kg   
2   I-2305               BACON PANCETTA      1.0      Kg     1.0     Kg   
3   I-1207              BAGUETTE FRENCH     24.0    each     1.0     CT   
4  I-17203               BALSAMIC GLAZE      2.0  bottle     2.0      L   

   InventoryGroup  
0         PRODUCE  
1         PRODUCE  
2            MEAT  
3           BREAD  
4  FOOD - GROCERY

In [7]:
Items.shape

(598, 7)

In [8]:
# Read Ingredients_List.csv
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [9]:
Ingredients.head()

IngredientId   Qty    Uom  Conversion  InvFactor   Recipe
0      P-18746  1.00     Kg     1.00000     1.0000  P-10241
1       I-3388  1.00      L     1.00000     0.3058  P-10496
2       I-4660  2.27     Kg     2.20462     0.6942  P-10496
3       I-3451  2.56      L     1.00000     1.2800  P-13933
4       I-4679  1.00  BUNCH     1.00000     0.0063  P-18318

In [10]:
Ingredients.shape

(5382, 6)

In [11]:
# Read Preps_List.csv
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [12]:
Preps.head()

PrepId                  Description  PakQty PakUOM InventoryGroup
0  P-55516  BAKED|Lasagna|Spin Mushroom   5.550     Kg            NaN
1  P-54666  BAKED|Pasta|Chicken Alfredo   6.176     Kg            NaN
2  P-54664    BAKED|Pasta|Chorizo Penne   7.360     Kg            NaN
3  P-56502     BAKED|Pasta|Shrimp Pesto   5.760     Kg            NaN
4  P-56433       BATCH|Shrimp Remoulade   1.600     Kg            NaN

In [13]:
Preps.shape

(753, 5)

In [14]:
# Read Product_List.csv
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [15]:
Products.head()

ProdId                     Description     SalesGroup
0  R-61778          ALF|Flatbread|4 Cheese  OK - AL FORNO
1  R-61780  ALF|Flatbread|Apple & Pancetta  OK - AL FORNO
2  R-61749       ALF|Flatbread|BBQ Chicken  OK - AL FORNO
3  R-50859        ALF|Flatbread|Bruschetta  OK - AL FORNO
4  R-50788           ALF|Flatbread|Caprese  OK - AL FORNO

In [16]:
Products.shape

(458, 3)

In [17]:
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [18]:
Conversions.head()

ConversionId  Multiplier  ConvertFromQty ConvertFromUom  ConvertToQty  \
0          NaN    1.000000             1.0            XXX          1.00   
1          NaN    0.877193             1.0          1.14L          1.14   
2          NaN    0.666667             1.0           1.5L          1.50   
3          NaN    0.571429             1.0         1.75 L          1.75   
4          NaN    0.500000             1.0             2L          2.00   

  ConvertToUom  
0            L  
1            L  
2            L  
3            L  
4            L

In [19]:
Conversions.shape

(296, 6)

***
## Update Conversion List

In [20]:
# Add the specific conversion info from the newly-processed data to a unit conversion database
Update_Conv = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Conv_UpdateConv.csv"))
Update_Conv

ConversionId  Multiplier  ConvertFromQty ConvertFromUom  ConvertToQty  \
0         I-1028    0.008333             1.0             CT         120.0   
1         I-1034    0.008333             1.0             CT         120.0   
2         I-1035    0.010000             1.0             CT         100.0   
3        I-10605    0.008850             1.0             CT         113.0   
4         I-1126    0.006667             1.0             CT         150.0   
..           ...         ...             ...            ...           ...   
310      P-32664    0.016393             1.0           each          61.0   
311      P-55707    0.005405             1.0            PTN         185.0   
312      P-55709    0.005405             1.0            PTN         185.0   
313      P-62293    0.005405             1.0            PTN         185.0   
314      P-62023    0.006452             1.0           ROLL         155.0   

    ConvertToUom  
0              g  
1              g  
2              g  
3              g  
4              g  
..           ...  
310            g  
311            g  
312            g  
313            g  
314            g  

[315 rows x 6 columns]

In [21]:
for index, row in Update_Conv.iterrows():
    Id = Update_Conv.loc[index, 'ConversionId']
    Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace = True)

In [22]:
frames = [Conversions, Update_Conv]
Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [23]:
Conversions

ConversionId  Multiplier  ConvertFromQty ConvertFromUom  ConvertToQty  \
0            NaN    1.000000             1.0            XXX          1.00   
1            NaN    0.877193             1.0          1.14L          1.14   
2            NaN    0.666667             1.0           1.5L          1.50   
3            NaN    0.571429             1.0         1.75 L          1.75   
4            NaN    0.500000             1.0             2L          2.00   
..           ...         ...             ...            ...           ...   
585      P-32664    0.016393             1.0           each         61.00   
586      P-55707    0.005405             1.0            PTN        185.00   
587      P-55709    0.005405             1.0            PTN        185.00   
588      P-62293    0.005405             1.0            PTN        185.00   
589      P-62023    0.006452             1.0           ROLL        155.00   

    ConvertToUom  
0              L  
1              L  
2              L  
3              L  
4              L  
..           ...  
585            g  
586            g  
587            g  
588            g  
589            g  

[590 rows x 6 columns]

In [24]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
Conversions.to_csv(path, index = False, header = True)

### Create Unit Converter

In [25]:
# Import standard unit conversion information and construct a dataframe
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

Multiplier  ConvertFromQty ConvertFromUom  ConvertToQty ConvertToUom
0     4.92890               1            tsp       4.92890           ml
1    14.78700               1           Tbsp      14.78700           ml
2   946.35000               1             qt     946.35000           ml
3   473.17625               1             pt     473.17625           ml
4    28.34950               1             oz      28.34950            g

In [26]:
# Seperate uoms that converted to 'ml' or 'g'
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [27]:
# Construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [28]:
# Test the std_converter
#assert std_converter(0.25,'lb') == (113.398, 'g')

In [29]:
# Construct a unit converter for specific ingredients
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [30]:
# Test the spc_converter
#assert spc_converter('I-1120', 1, 'CT') == (50, 'g')

***
## Items with Non-standard Units

In [31]:
# Filter out the items whose unit information is unknown 
col_names = list(Ingredients.columns.values)
Items_Nonstd = []

for index, row in Ingredients.iterrows():
    Ingre = Ingredients.loc[index,'IngredientId']
    Uom = Ingredients.loc[index,'Uom']
    if Uom not in ['g', 'ml'] and Uom not in liquid_unit + solid_unit and Ingre.startswith('I') and Ingre not in Conversions["ConversionId"].tolist():
        Dict = {}
        Dict.update(dict(row))
        Items_Nonstd.append(Dict)

Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
Items_Nonstd.drop_duplicates(subset=['IngredientId'], inplace=True,)
Items_Nonstd

Empty DataFrame
Columns: [IngredientId, Qty, Uom, Conversion, InvFactor, Recipe]
Index: []

In [32]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Items_Nonstd.csv")
Items_Nonstd.to_csv(path, index = False, header = True)

***
## Clean Preps Units

In [33]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [34]:
# Convert uom into 'g' or 'ml' for each prep using the unit converter
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

In [35]:
Preps

PrepId                  Description   PakQty PakUOM InventoryGroup  \
0    P-55516  BAKED|Lasagna|Spin Mushroom    5.550     Kg            NaN   
1    P-54666  BAKED|Pasta|Chicken Alfredo    6.176     Kg            NaN   
2    P-54664    BAKED|Pasta|Chorizo Penne    7.360     Kg            NaN   
3    P-56502     BAKED|Pasta|Shrimp Pesto    5.760     Kg            NaN   
4    P-56433       BATCH|Shrimp Remoulade    1.600     Kg            NaN   
..       ...                          ...      ...    ...            ...   
748  P-47418                   MIX|Cheese    2.000     Kg           PREP   
749  P-42317     ROASTED|Spaghetti Squash    1.400     Kg            NaN   
750  P-56927       SAUTE|Cauliflower Rice    1.000     Kg            NaN   
751  P-56887            YIELD|Grated Pear  800.000      g            NaN   
752  P-46509            YIELD|Lettuce bun    3.000    PTN            NaN   

          StdQty StdUom  
0    5550.000000      g  
1    6176.000000      g  
2    7360.000000      g  
3    5760.000000      g  
4    1600.000000      g  
..           ...    ...  
748  2000.000000      g  
749  1400.000000      g  
750  1000.000000      g  
751   800.000000      g  
752   449.999978      g  

[753 rows x 7 columns]

In [36]:
# Save cleaned preps list to file
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

### Get Preps with Nonstandard Unit

In [37]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [38]:
Preps_Nonstd

PrepId            Description  PakQty PakUOM  InventoryGroup  StdQty  \
0  P-51230  WINGS|Sauce Mix|ratio     1.0   srvg             NaN     1.0   

  StdUom  
0   srvg

In [39]:
# Filter out preps with nonstandard uom but have information already
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))

col_names = list(Preps_Nonstd.columns.values)
Preps_Nonstd_na = []

for index, row in Preps_Nonstd.iterrows():
    PrepId = Preps_Nonstd.loc[index,'PrepId']
    if PrepId not in Manual_PrepU['PrepId'].values:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd_na.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd_na, columns = col_names)
Preps_Nonstd

Empty DataFrame
Columns: [PrepId, Description, PakQty, PakUOM, InventoryGroup, StdQty, StdUom]
Index: []

In [40]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

***

## New Items

In [41]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

ItemId  CategoryID               Description  CaseQty CaseUOM  PakQty  \
0  I-57545           1   CHUCK FLAT BONELESS FZN     3.30      Kg     1.0   
1  I-10869           1       BEEF STIRFRY COV FR     5.00      Kg     1.0   
2   I-7064           1     BEEF OUTSIDE FLAT AAA     1.00      Kg     1.0   
3  I-37005           1            BEEF MEATBALLS     4.54      Kg  1000.0   
4  I-37002           1  BEEF INSIDE ROUND SHAVED     9.00      Kg  1000.0   

  PakUOM InventoryGroup  
0     Kg           MEAT  
1     Kg           MEAT  
2     Kg           MEAT  
3      g           MEAT  
4      g           MEAT

In [42]:
Items_Assigned.shape

(1993, 8)

### Get the List of New Items

In [43]:
# Filter new items by itemID that not in the database and output them in a dataframe
col_names = list(Items.columns.values)
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [44]:
New_Items.insert(1, "CategoryID", '')
New_Items

Empty DataFrame
Columns: [ItemId, CategoryID, Description, CaseQty, CaseUOM, PakQty, PakUOM, InventoryGroup]
Index: []

In [45]:
New_Items.shape

(0, 8)

In [46]:
# Store the list of new items into .csv file
if not New_Items.empty:
    path = os.path.join(os.getcwd(), "data", "mapping", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
    New_Items.to_csv(path, index = False, header = True)

***
## Data Summary

In [47]:
datasum = pd.DataFrame([New_Items.shape, Preps_Nonstd.shape, Items_Nonstd.shape],
                       columns = ['count', 'columns'], 
                       index = ['New_Items', 'Preps_Nonstd', 'Items_Nonstd'])
datasum

count  columns
New_Items         0        8
Preps_Nonstd      0        7
Items_Nonstd      0        6